In [4]:
# -*- coding: utf-8 -*-
import copy
import random
import pandas as pd
import networkx as nx
from tnm import *
import datetime

时变网络的数据格式如下：
第一列表示节点1，第二列表示节点2，第三列表示时间戳。

1 	2 	1000000
3 	4 	1000003
5 	6 	1000007
2 	1 	1000032
7 	8 	1000035
9 	10 	1000035
11 	12 	1000036
13 	14 	1000038
15 	16 	1000048
1 	2 	1000107
17 	18 	1000120 


显然同一对节点（即一条边）上可能出现多个不同的时间戳（如上图中第1行和第10行表示节点1和节点2在两个不同时间戳处有交互），故用以往构建静态无权/加权网络的方式构建时变网络是不可行的，所以我们采用多重变图进行网络的构建。

In [2]:
filename = "SD03.txt"     
#读入数据
fh = pd.read_csv(filename,names=['node1', 'node2', 'timestamp'],header=None,delim_whitespace=True)
fh.head(20)

,node1,node2,timestamp
0,1,2,1000000
1,3,4,1000003
2,5,6,1000007
3,2,1,1000032
4,7,8,1000035
5,9,10,1000035
6,11,12,1000036
7,13,14,1000038
8,15,16,1000048
9,1,2,1000107


In [3]:
#构建网络 
G=nx.from_pandas_dataframe(fh,'node1','node2',edge_attr='timestamp', create_using=nx.MultiGraph())
# 若报错为TypeError: unhashable type: 'dict'
# 需要修改Anaconda2\lib\site-packages\networkx\convert_matrix.py 211行
# 将g.add_edge(row[src_i], row[tar_i], {i:row[j] for i, j in edge_i})
# 改为g.add_edge(row[src_i], row[tar_i], attr_dict={i:row[j] for i, j in edge_i})
# 修改后重启核 运行即可

由于我们采用多重图方式进行时变网络的构建，所以断边重连和时序交换等需要按多重图的网络结构进行。在多重图中两节点间有多条边存在，G.remove_edge(u,v)并非删掉u-v间的所有连边而是随机删掉一条，所以以下置乱算法凡需要断边均采用G.remove_edge(u,v,key=i)对指定边进行删除。

### 0阶连边置乱
>思想：从网络中随机选一条边和两个不相连的节点，断边重连，且新连边时序等于断开的那条边的时序

具体实现：
while swapcount < nswap:
1.	任选一条边u-v和两个不相连的节点x、y，
2.	连新边x-y并赋u-v边上的时间戳，断开边u-v，swapcount+=1


In [4]:
#时变网络的0阶连边置乱算法
def edges_swap_0k(G0, nswap=1, max_tries=100):   
    """
    从网络中随机选一条边和两个不相连的节点，断边重连，且新连边时序等于断开的那条边的时序  
    """
    if nswap > max_tries:
        raise nx.NetworkXError("Number of swaps > number of tries allowed.")
    if len(G0) < 3:
        raise nx.NetworkXError("Graph has less than three nodes.")
    
    n = 0
    swapcount = 0
    G = copy.deepcopy(G0)
    nodes = G.nodes()
    
    while swapcount < nswap:
        if n >= max_tries:
            e=('Maximum number of swap attempts (%s) exceeded '%n +
            'before desired swaps achieved (%s).'%nswap)
            print e
            break
        n += 1
        x,u = random.sample(nodes,2)    #随机找两个节点
        candicate_y = [no for no in nodes if no not in list(G[x])]
        if list(G[u]) and candicate_y:
            v = random.choice(list(G[u])) #任选一条边u-v
            y = random.choice(candicate_y)  #任选一条不相连边x-y
        else:
            continue
        if len(set([u,v,x,y])) < 3: #防止自环           
            continue
        for i in range(G.number_of_edges(u,v)):
            G.add_edge(x,y,key=i,timestamp=G[u][v][i]['timestamp'])            
            G.remove_edge(u,v,key=i)       
        swapcount+=1
        
    return G

### 1阶连边置乱
>思想：随机取两条边 u-v 和 x-y, 且节点u和x,v和y无连边, 则断边重连且t(u,x)=t(u,v)及t(v,y)=t(x,y)

具体实现：
while swapcount < nswap:
1.	任选两条边u-v，x-y，
2.	若u-y或x-v有连边，跳回步骤1，反之执行步骤3
3.	连新边u-y并赋u-v边上的时间戳，断开边u-v，同理连x-v, 断x-y，swapcount+=1

In [5]:
#时变网络的1阶连边置乱算法
def edges_swap_1k(G0, nswap=1, max_tries=100):  
    """
    随机取两条边 u-v 和 x-y, 且节点u和x,v和y无连边, 则断边重连且t(u,x)=t(u,v)及t(v,y)=t(x,y)
    """
    if nswap>max_tries:
        raise nx.NetworkXError("Number of swaps > number of tries allowed.")
    if len(G0) < 4:
        raise nx.NetworkXError("Graph has less than four nodes.")
    
    n=0
    swapcount=0
    G = copy.deepcopy(G0)
    keys,degrees=zip(*G.degree().items()) 
    cdf=nx.utils.cumulative_distribution(degrees)
    
    while swapcount < nswap:
        if n >= max_tries:
            e=('Maximum number of swap attempts (%s) exceeded '%n +
            'before desired swaps achieved (%s).'%nswap)
            print e
            break
        n+=1
        (ui,xi)=nx.utils.discrete_sequence(2,cdistribution=cdf)
        if ui==xi:
            continue 
        u=keys[ui] 
        x=keys[xi]
        v=random.choice(list(G[u]))
        y=random.choice(list(G[x]))        
        if len(set([u,v,x,y])) < 4: #防止自环           
            continue
        if y not in G[u] and v not in G[x]: 
            for i in range(G.number_of_edges(u,v)):
                G.add_edge(u,y,key=i,timestamp=G[u][v][i]['timestamp'])
                G.remove_edge(u,v,key=i)
            for j in range(G.number_of_edges(x,y)):
                G.add_edge(x,v,key=j,timestamp=G[x][y][j]['timestamp']) 
                G.remove_edge(x,y,key=i)
#            G.remove_edges_from([(u,v),(x,y)])#仅移除一条边
            swapcount+=1        
    return G


### 保持个体天、月、周模式的时间置乱算法
>思想：在保证单条边上不出现相同时间戳的前提下，分别从两条边上任选一个时间戳（具有相同的天/月/周模式）互换

具体实现：
while swapcount < nswap:
1.	任选两条边u-v，x-y，并分别从其上任选一个时间戳stamp1，stamp2
2.	若stamp1在u-v边上存在或stamp2在x-y边上存在或stamp1.to_day == stamp2.to_day(周月同理)，跳回步骤1，反之执行步骤3
3.	时间戳stamp1，stamp2互换，swapcount+=1

In [17]:
# G.edges_iter(data=True).next()
stamp1 = G[1][512][1]['timestamp']
stamp1_date = datetime.datetime.utcfromtimestamp(stamp1)
stamp1_date.month

1

In [19]:

#保持个体天、月、周模式的时间置乱算法
def time_SameMode_swap(G0, nswap=1, max_tries=100, mode='day'):  
    """
    在保证单条边上不出现相同时间戳的前提下，分别从两条边上任选一个时间戳（具有相同的天/月/周模式）互换
    """
    if nswap>max_tries:
        raise nx.NetworkXError("Number of swaps > number of tries allowed.")
    if len(G0) < 3:
        raise nx.NetworkXError("Graph has less than three nodes.")
    
    n=0
    swapcount=0
    G = copy.deepcopy(G0)
    keys,degrees=zip(*G.degree().items()) 
    cdf=nx.utils.cumulative_distribution(degrees)
    while swapcount < nswap:
        if n >= max_tries:
            e=('Maximum number of swap attempts (%s) exceeded '%n +
            'before desired swaps achieved (%s).'%nswap)
            print e
            break
        n+=1
        (ui,xi)=nx.utils.discrete_sequence(2,cdistribution=cdf)
        if ui==xi:
            continue 
        u=keys[ui] 
        x=keys[xi]
        v=random.choice(list(G[u]))
        y=random.choice(list(G[x]))
        if len(set([u,v,x,y])) < 3: #保证从两条边上选时间戳           
            continue
        key1 = random.choice(range(G.number_of_edges(u,v)))
        key2 = random.choice(range(G.number_of_edges(x,y)))
        stamp1 = G[u][v][key1]['timestamp']
        stamp2 = G[x][y][key2]['timestamp']
        #Python下将时间戳转换到日期
        stamp1_date = datetime.datetime.utcfromtimestamp(stamp1)
        stamp2_date = datetime.datetime.utcfromtimestamp(stamp2)
        if mode == 'day': #保持个体天模式
            if stamp1_date.day != stamp2_date.day or {'timestamp':stamp1} in G[x][y].values() or {'timestamp':stamp2} in G[u][v].values():
                continue
        elif mode == 'week': #保持个体周模式
            if stamp1_date.weekday() != stamp2_date.weekday() or {'timestamp':stamp1} in G[x][y].values() or {'timestamp':stamp2} in G[u][v].values():
                continue
        elif mode == 'month': #保持个体月模式
            if stamp1_date.month != stamp2_date.month or {'timestamp':stamp1} in G[x][y].values() or {'timestamp':stamp2} in G[u][v].values():
                continue
        else:
            G[u][v][key1]['timestamp'] = stamp2
            G[x][y][key2]['timestamp'] = stamp1
        swapcount+=1        
    return G

### 时间置乱算法
>思想：在保证单条边上不出现相同时间戳的前提下，分别从两条边上任选一个时间戳互换

具体实现：
while swapcount < nswap:
1.	任选两条边u-v，x-y，并分别从其上任选一个时间戳stamp1，stamp2
2.	若stamp1在u-v边上存在或stamp2在x-y边上存在或stamp1 == stamp2，跳回步骤1，反之执行步骤3
3.	时间戳stamp1，stamp2互换，swapcount+=1

In [6]:
#时变网络的时间置乱算法
def time_swap(G0, nswap=1, max_tries=100):  
    """
    在保证单条边上不出现相同时间戳的前提下，分别从两条边上任选一个时间戳互换
    """
    if nswap>max_tries:
        raise nx.NetworkXError("Number of swaps > number of tries allowed.")
    if len(G0) < 3:
        raise nx.NetworkXError("Graph has less than three nodes.")
    
    n=0
    swapcount=0
    G = copy.deepcopy(G0)
    keys,degrees=zip(*G.degree().items()) 
    cdf=nx.utils.cumulative_distribution(degrees)
    while swapcount < nswap:
        if n >= max_tries:
            e=('Maximum number of swap attempts (%s) exceeded '%n +
            'before desired swaps achieved (%s).'%nswap)
            print e
            break
        n+=1
        (ui,xi)=nx.utils.discrete_sequence(2,cdistribution=cdf)
        if ui==xi:
            continue 
        u=keys[ui] 
        x=keys[xi]
        v=random.choice(list(G[u]))
        y=random.choice(list(G[x]))
        if len(set([u,v,x,y])) < 3: #保证从两条边上选时间戳           
            continue
        key1 = random.choice(range(G.number_of_edges(u,v)))
        key2 = random.choice(range(G.number_of_edges(x,y)))
        stamp1 = G[u][v][key1]['timestamp']
        stamp2 = G[x][y][key2]['timestamp']
        if stamp1 == stamp2 or {'timestamp':stamp1} in G[x][y].values() or {'timestamp':stamp2} in G[u][v].values():
            continue
        else:
            G[u][v][key1]['timestamp'] = stamp2
            G[x][y][key2]['timestamp'] = stamp1
        swapcount+=1        
    return G
    

### 时间随机化置乱算法
>思想：在保证单条边上不出现相同时间戳的前提下，任选一个时间戳用在该网络时间范围内的新生时间戳代替

具体实现：
while swapcount < nswap:
1.	任选一条边u-v及其上的一个时间戳stamps1，在该网络时间范围内的新生时间戳stamp2
2.	若stamp1 == stamp2 或 u-v 边上已存在时间戳stamp2 ，执行步骤1，反之执行步骤3
3.	将u-v边上的时序戳stamp1用stamp2替换。
swapcount+=1


In [7]:
#时变网络的时间随机化算法
def time_random(G0, nswap=1, max_tries=100,mins=1000000,maxs=31556926):  
    """
    在保证单条边上不出现相同时间戳的前提下，任选一个时间戳用在该网络时间范围内的新生时间戳代替
    """
    if nswap>max_tries:
        raise nx.NetworkXError("Number of swaps > number of tries allowed.")
    if len(G0) < 3:
        raise nx.NetworkXError("Graph has less than three nodes.")
    
    n=0
    swapcount=0
    G = copy.deepcopy(G0)
    while swapcount < nswap:
        if n >= max_tries:
            e=('Maximum number of swap attempts (%s) exceeded '%n +
            'before desired swaps achieved (%s).'%nswap)
            print e
            break
        n+=1
        u,v = random.choice(G.edges())
        key1 = random.choice(range(G.number_of_edges(u,v)))
        stamp1 = G[u][v][key1]['timestamp']
        stamp2 = random.randrange(mins,maxs)
        if stamp1 == stamp2 or {'timestamp':stamp2} in G[u][v].values():
            continue
        else:
            G[u][v][key1]['timestamp'] = stamp2
        swapcount+=1        
    return G


### 时权置乱算法
>思想：任选两条边，将其上的时间序列互换

具体实现：
while swapcount < nswap:
1.	任选两条边u-v，x-y 
2.	将u-v边上的时序存到列表stamp1中并移除边u-v，x-y边上的时序存到列表stamp2中并移除边x-y。然后遍历stamp1,将时序添加到x-y边上，同理将stamp2时序添加到边u-v上。
swapcount+=1


In [8]:
#时变网络的时权置乱算法
def timeweight_swap(G0, nswap=1, max_tries=100):  
    """
    任选两条边，将其上的时间序列互换
    """
    if nswap>max_tries:
        raise nx.NetworkXError("Number of swaps > number of tries allowed.")
    if len(G0) < 3:
        raise nx.NetworkXError("Graph has less than three nodes.")
    
    n=0
    swapcount=0
    G = copy.deepcopy(G0)
    keys,degrees=zip(*G.degree().items()) 
    cdf=nx.utils.cumulative_distribution(degrees)
    while swapcount < nswap:
        if n >= max_tries:
            e=('Maximum number of swap attempts (%s) exceeded '%n +
            'before desired swaps achieved (%s).'%nswap)
            print e
            break
        n+=1
        (ui,xi)=nx.utils.discrete_sequence(2,cdistribution=cdf)
        if ui==xi:
            continue 
        u=keys[ui] 
        x=keys[xi]
        v=random.choice(list(G[u]))
        y=random.choice(list(G[x]))
        if len(set([u,v,x,y])) < 3: #保证从两条边上选          
            continue
        stamp1 = []
        stamp2 = []        
        print G.number_of_edges(u,v),G.number_of_edges(x,y)
        for i in range(G.number_of_edges(u,v)):
            stamp1.append(G[u][v][i]['timestamp'])
            G.remove_edge(u,v,key=i)
        for i in range(G.number_of_edges(x,y)):
            stamp2.append(G[x][y][i]['timestamp'])
            G.remove_edge(x,y,key=i)
        print G.number_of_edges(u,v),G.number_of_edges(x,y)
        for i in range(len(stamp2)):
            G.add_edge(u,v,key=i,timestamp=stamp2[i])
        for i in range(len(stamp1)):
            G.add_edge(x,y,key=i,timestamp=stamp1[i])
        print G.number_of_edges(u,v),G.number_of_edges(x,y)
        swapcount+=1        
    return G

### 等时权置乱算法
>思想：任选两条权重相等的边，将其上的时间序列互换

具体实现：
while swapcount < nswap:
1.	任选两条边u-v，x-y 
2.	若这两条边权重相等，将u-v边上的时序存到列表stamp1中并移除边u-v，x-y边上的时序存到列表stamp2中并移除边x-y。然后遍历stamp1,将时序添加到x-y边上，同理将stamp2时序添加到边u-v上。
swapcount+=1
注：步骤2为了和时权置乱保持一致性，亦可不断边重连完成时序互换。

In [9]:
#时变网络的等时权置乱算法
def sametimeweight_swap(G0, nswap=1, max_tries=100):  
    """
    任选两条权重相等的边，将其上的时间序列互换
    """
    if nswap>max_tries:
        raise nx.NetworkXError("Number of swaps > number of tries allowed.")
    if len(G0) < 3:
        raise nx.NetworkXError("Graph has less than three nodes.")
    
    n=0
    swapcount=0
    G = copy.deepcopy(G0)
    keys,degrees=zip(*G.degree().items()) 
    cdf=nx.utils.cumulative_distribution(degrees)
    while swapcount < nswap:
        if n >= max_tries:
            e=('Maximum number of swap attempts (%s) exceeded '%n +
            'before desired swaps achieved (%s).'%nswap)
            print e
            break
        n+=1
        (ui,xi)=nx.utils.discrete_sequence(2,cdistribution=cdf)
        if ui==xi:
            continue 
        u=keys[ui] 
        x=keys[xi]
        v=random.choice(list(G[u]))
        y=random.choice(list(G[x]))
        if len(set([u,v,x,y])) < 3: #保证从两条边上选          
            continue
        if G.number_of_edges(u,v) == G.number_of_edges(x,y):#两条边权重相等
            stamp1 = []
            stamp2 = []        
            print G.number_of_edges(u,v),G.number_of_edges(x,y)
            for i in range(G.number_of_edges(u,v)):
                stamp1.append(G[u][v][i]['timestamp'])
                G.remove_edge(u,v,key=i)
            for i in range(G.number_of_edges(x,y)):
                stamp2.append(G[x][y][i]['timestamp'])
                G.remove_edge(x,y,key=i)
            print G.number_of_edges(u,v),G.number_of_edges(x,y)
            for i in range(len(stamp2)):
                G.add_edge(u,v,key=i,timestamp=stamp2[i])
            for i in range(len(stamp1)):
                G.add_edge(x,y,key=i,timestamp=stamp1[i])
            print G.number_of_edges(u,v),G.number_of_edges(x,y)
            swapcount+=1        
    return G

### 接触置乱算法
>思想：在保证单条边上不出现相同时间戳的前提下，将时间戳和连边随机组合

具体实现：
while swapcount < nswap:
1.	任选两条边u-v，x-y，并从u-v上任选一个时间戳stamp1，
2.	若该时间戳在x-y边上存在或u-v边上仅有这一个时间戳，跳回步骤1，反之执行步骤3
3.	去掉时间戳为stamp1的边u-v，增加一条时间戳为stamp1的边x-y，swapcount+=1

有这么一种情形 即存在这么一些时间戳，节点1给节点2发消息的同时节点2也给节点1发消息，那这样我们取边上的接触时间戳就对这种情形只取一次（因为要求单条边上时间戳不同） 临时想到的，有向的话肯定不会有问题，无向不知道会不会对算法有影响 
上述数据处理后应该变成1	2	[1,2,3]还是1	2	[1,3]


In [10]:
#时变网络的接触置乱算法
def time_randomswap(G0, nswap=1, max_tries=100):  
    """
    在保证单条边上不出现相同时间戳的前提下，将时间戳和连边随机组合
    """
    if nswap>max_tries:
        raise nx.NetworkXError("Number of swaps > number of tries allowed.")
    if len(G0) < 3:
        raise nx.NetworkXError("Graph has less than three nodes.")
    
    n=0
    swapcount=0
    G = copy.deepcopy(G0)
    keys,degrees=zip(*G.degree().items()) 
    cdf=nx.utils.cumulative_distribution(degrees)
    while swapcount < nswap:
        if n >= max_tries:
            e=('Maximum number of swap attempts (%s) exceeded '%n +
            'before desired swaps achieved (%s).'%nswap)
            print e
            break
        n+=1
        #任选两条边u-v，x-y
        (ui,xi)=nx.utils.discrete_sequence(2,cdistribution=cdf)
        if ui==xi:
            continue 
        u=keys[ui] 
        x=keys[xi]
        v=random.choice(list(G[u]))
        y=random.choice(list(G[x]))
        if len(set([u,v,x,y])) < 3: #保证从两条边上选          
            continue
        #从u-v上任选一个时间戳
        key1 = random.choice(range(G.number_of_edges(u,v)))
        stamp1 = G[u][v][key1]['timestamp']
        if {'timestamp':stamp1} in G[x][y].values() or G.number_of_edges(u,v) < 2:
            continue
        else:
            G.remove_edge(u,v,key=key1)
            G.add_edge(x,y,key=G.number_of_edges(x,y),timestamp=stamp1)
            swapcount+=1        
    return G

### 时间倒转算法
>思想： 把连边上的接触时间倒转过来

具体实现：<br>
1、读入文件fh后,按时间戳大小排序，得到有序dataframe数据fh1<br>
2、翻转fh1['timestamp']列<br>
3、将翻转后的数据组成网络<br>

In [27]:
fh1 = fh.sort_values(by='timestamp')
ts_list = list(fh1['timestamp'])
ts_list.reverse()
fh1['timestamp'] = ts_list
fh1.head()

,node1,node2,timestamp
0,1,2,30231938
1,3,4,30231933
2,5,6,30231921
3,2,1,30231907
4,7,8,30231857


In [30]:
#时间倒转后的网络
def time_reverse(fh):
    fh1 = fh.sort_values(by='timestamp')
    ts_list = list(fh1['timestamp'])
    ts_list.reverse()
    fh1['timestamp'] = ts_list
    return fh1
G_reverse=nx.from_pandas_dataframe(time_reverse(fh),'node1','node2',edge_attr='timestamp', create_using=nx.MultiGraph())

In [ ]:
nswap = 2 * len(G.edges())
maxtry = 100 * nswap

#置乱网络
G0 = edges_swap_0k(G, nswap=nswap, max_tries=maxtry)
G1 = edges_swap_1k(G, nswap=nswap, max_tries=maxtry)

#时变网络的时间置乱算法
G1 = time_swap(G, nswap=1, max_tries=maxtry)

#时变网络的时间随机化算法
mins = min(fh['timestamp'])
maxs = max(fh['timestamp'])
G1 = time_random(G, nswap=nswap, max_tries=maxtry,mins=mins,maxs=maxs)

#时变网络的时权置乱算法
G1 = timeweight_swap(G, nswap=1, max_tries=maxtry)

#时变网络的等时权置乱算法
G1 = sametimeweight_swap(G, nswap=1, max_tries=maxtry)

#时变网络的接触置乱算法
G1 = time_randomswap(G, nswap=nswap, max_tries=maxtry)